In [1]:
import os
import subprocess

import numpy as np

from hyperopt import hp
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform, conditional

import tensorflow as tf
import keras
from keras import backend as K
from keras.datasets import mnist
from keras.layers import BatchNormalization, Flatten, Dense
from keras.layers import Input, Conv2D, Convolution2D
from keras.layers import Activation, concatenate, Dropout
from keras.layers import AveragePooling2D, GlobalAveragePooling2D, MaxPooling2D
from keras.models import Sequential, Model
from keras.utils import np_utils, to_categorical
from keras.applications.imagenet_utils import _obtain_input_shape

import Augmentor
import json

import tuner
from tuner import utils
from tuner import load_data
from tuner import augment_data
from tuner import use_hyperas
from tuner import net
from tuner.dataset import ClassificationDataset, AugmentDataset

Using TensorFlow backend.
/home/fytroo/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
from easydict import EasyDict
params = EasyDict({
    'src_dir': '../micin-dataset/brain',
    'dst_dir': './brain'
})

### Convert  dataset from brain-dir to ready-dir 

In [3]:
df = load_data.df_fromdir_brain(params.src_dir)

In [4]:
load_data.classed_dir_fromdf(df, "tmp/brain")

### Load dataset via ready-dir

In [5]:
brain = ClassificationDataset('tmp/brain')

In [6]:
brain_dataset = brain

In [7]:
min(brain_dataset.counts_train_data().values())

46

In [8]:
aug_brain = AugmentDataset(brain_dataset)

Initialised with 246 image(s) found.
Output directory set to /home/fytroo/Tuner/dev/standard_datasets/5a66ca2cb037cb158074073c/train/output.

In [9]:
aug_brain.augmented_dir

'/home/fytroo/Tuner/dev/standard_datasets/5a66ca2cb037cb158074073c/auged'

In [10]:
aug_brain.dataset.train_dir

'/home/fytroo/Tuner/dev/standard_datasets/5a66ca2cb037cb158074073c/train'

### Search best condition of data augmentation

In [11]:
aug_brain.search_opt_augment(model=net.neoaug)

def data():
    train_dir = '/home/fytroo/Tuner/dev/standard_datasets/5a66ca2cb037cb158074073c/train' 
    test_dir = '/home/fytroo/Tuner/dev/standard_datasets/5a66ca2cb037cb158074073c/validation'
    resize = 96 
    rescale = 1 
    df = load_data.df_fromdir_classed(train_dir)
    x_train, y_train = load_data.load_fromdf(df, resize=resize, rescale=rescale)
    df = load_data.df_fromdir_classed(test_dir)
    x_test, y_test = load_data.load_fromdf(df, resize=resize, rescale=rescale)

    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)

    return x_train, y_train, x_test, y_test

         name label        handle  \
0   MS163.jpg    MS  MS/MS163.jpg   
1   MS139.jpg    MS  MS/MS139.jpg   
2   MS159.jpg    MS  MS/MS159.jpg   
3   MS146.jpg    MS  MS/MS146.jpg   
4   MS118.jpg    MS  MS/MS118.jpg   
5   PD044.jpg    PD  PD/PD044.jpg   
6   PD057.jpg    PD  PD/PD057.jpg   
7   PD111.jpg    PD  PD/PD111.jpg   
8   PD062.jpg    PD  PD/PD062.jpg   
9   PD105.jpg    PD  

Epoch 5/10
 - 0s - loss: 1.3676 - acc: 0.3527 - val_loss: 10.6017 - val_acc: 0.2500
Epoch 6/10
 - 0s - loss: 1.3519 - acc: 0.3705 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 7/10
 - 0s - loss: 1.3170 - acc: 0.3929 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 8/10
 - 0s - loss: 1.3529 - acc: 0.3705 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 9/10
 - 0s - loss: 1.3536 - acc: 0.3884 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 10/10
 - 0s - loss: 1.3449 - acc: 0.3795 - val_loss: 12.0886 - val_acc: 0.2500
Test accuracy: 0.25
--------------------------------------------------------------------------------
Operations: 3
	0: Flip (probability=0.5 top_bottom_left_right=LEFT_RIGHT )
	1: RandomErasing (probability=0.5 rectangle_area=0.2 )
	2: Shear (probability=0.3 max_shear_left=2 max_shear_right=2 )
Images: 0
Classes: 0

You can remove operations using the appropriate index and the remove_operation(index) function.
Epoch 1/10
 - 0s - loss: 1.4355 - acc: 0.3304 - val_loss: 9.8076 - val_acc: 0.350

Epoch 1/10
 - 1s - loss: 1.4499 - acc: 0.3036 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 2/10
 - 0s - loss: 1.3314 - acc: 0.3482 - val_loss: 11.2827 - val_acc: 0.3000
Epoch 3/10
 - 0s - loss: 1.3161 - acc: 0.4375 - val_loss: 9.6709 - val_acc: 0.4000
Epoch 4/10
 - 0s - loss: 1.3299 - acc: 0.4286 - val_loss: 8.6482 - val_acc: 0.4000
Epoch 5/10
 - 0s - loss: 1.2980 - acc: 0.4509 - val_loss: 8.8650 - val_acc: 0.4500
Epoch 6/10
 - 0s - loss: 1.2345 - acc: 0.4821 - val_loss: 10.4844 - val_acc: 0.3500
Epoch 7/10
 - 0s - loss: 1.2222 - acc: 0.5045 - val_loss: 5.6228 - val_acc: 0.6000
Epoch 8/10
 - 0s - loss: 1.1729 - acc: 0.5491 - val_loss: 5.4511 - val_acc: 0.6500
Epoch 9/10
 - 0s - loss: 1.0411 - acc: 0.6205 - val_loss: 4.8354 - val_acc: 0.7000
Epoch 10/10
 - 0s - loss: 0.9978 - acc: 0.6295 - val_loss: 6.4476 - val_acc: 0.6000
Test accuracy: 0.600000023842
--------------------------------------------------------------------------------
Operations: 2
	0: Flip (probability=0.5 top_bottom_left

### Execute data augmentation with best condition

In [12]:
aug_brain.augment_dataset()

Processing MS125.jpg:   3%|▎         | 6/184 [00:00<00:03, 46.73 Samples/s]

Initialised with 46 image(s) found.
Output directory set to /home/fytroo/Tuner/dev/standard_datasets/5a66ca2cb037cb158074073c/train/MS/output.

Processing PD116.jpg:   3%|▎         | 5/184 [00:00<00:05, 32.92 Samples/s]  

Initialised with 46 image(s) found.
Output directory set to /home/fytroo/Tuner/dev/standard_datasets/5a66ca2cb037cb158074073c/train/PD/output.

Processing N100.jpg:  10%|▉         | 18/184 [00:00<00:01, 96.37 Samples/s]  

Initialised with 79 image(s) found.
Output directory set to /home/fytroo/Tuner/dev/standard_datasets/5a66ca2cb037cb158074073c/train/N/output.

Processing PS107.jpg:   3%|▎         | 5/184 [00:00<00:05, 33.35 Samples/s] 

Initialised with 75 image(s) found.
Output directory set to /home/fytroo/Tuner/dev/standard_datasets/5a66ca2cb037cb158074073c/train/PS/output.

### Tuning hyper parameter of CNN

In [14]:
best_condition, best_model = use_hyperas.exec_hyperas(
    aug_brain.augmented_dir,
    aug_brain.dataset.validation_dir,
    net.simplenet,
    batch_size = 32,
    epochs=10,
    optimizer='adam'
)
fname = 'simplenet.hdf5'
best_model.save(fname)

def data():
    train_dir = '/home/fytroo/Tuner/dev/standard_datasets/5a66ca2cb037cb158074073c/auged' 
    test_dir = '/home/fytroo/Tuner/dev/standard_datasets/5a66ca2cb037cb158074073c/validation'
    resize = 96 
    rescale = 1 
    df = load_data.df_fromdir_classed(train_dir)
    x_train, y_train = load_data.load_fromdf(df, resize=resize, rescale=rescale)
    df = load_data.df_fromdir_classed(test_dir)
    x_test, y_test = load_data.load_fromdf(df, resize=resize, rescale=rescale)

    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)

    return x_train, y_train, x_test, y_test

         name label        handle  \
0   MS163.jpg    MS  MS/MS163.jpg   
1   MS139.jpg    MS  MS/MS139.jpg   
2   MS159.jpg    MS  MS/MS159.jpg   
3   MS146.jpg    MS  MS/MS146.jpg   
4   MS118.jpg    MS  MS/MS118.jpg   
5   PD044.jpg    PD  PD/PD044.jpg   
6   PD057.jpg    PD  PD/PD057.jpg   
7   PD111.jpg    PD  PD/PD111.jpg   
8   PD062.jpg    PD  PD/PD062.jpg   
9   PD105.jpg    PD  

Epoch 21/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 22/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 23/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 24/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 25/100
 - 0s - loss: 12.0667 - acc: 0.2514 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 26/100
 - 0s - loss: 12.0667 - acc: 0.2514 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 27/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 28/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 29/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 30/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 31/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 32/100
 - 0s - loss: 12.0886 - acc: 0

Epoch 15/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 16/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 17/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 18/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 19/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 20/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 21/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 22/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 23/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 24/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 25/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 26/100
 - 0s - loss: 12.0886 - acc: 0

Epoch 9/100
 - 1s - loss: 12.0448 - acc: 0.2527 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 10/100
 - 1s - loss: 12.0667 - acc: 0.2514 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 11/100
 - 1s - loss: 12.1543 - acc: 0.2459 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 12/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 13/100
 - 1s - loss: 12.0667 - acc: 0.2514 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 14/100
 - 1s - loss: 12.1762 - acc: 0.2446 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 15/100
 - 1s - loss: 12.1105 - acc: 0.2486 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 16/100
 - 1s - loss: 12.1105 - acc: 0.2486 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 17/100
 - 1s - loss: 12.0667 - acc: 0.2514 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 18/100
 - 1s - loss: 12.1324 - acc: 0.2473 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 19/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 20/100
 - 1s - loss: 12.0229 - acc: 0.

Epoch 3/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 4/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 5/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 6/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 7/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 8/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 9/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 10/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 11/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 12/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 13/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 14/100
 - 1s - loss: 12.0886 - acc: 0.2500 -

Epoch 98/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 99/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 100/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Test accuracy: 0.25
Train on 736 samples, validate on 20 samples
Epoch 1/100
 - 2s - loss: 12.2725 - acc: 0.2364 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 2/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 3/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 4/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 5/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 6/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 7/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 8/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - v

Epoch 92/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 93/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 94/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 95/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 96/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 97/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 98/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 99/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 100/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Test accuracy: 0.25
Train on 736 samples, validate on 20 samples
Epoch 1/100
 - 2s - loss: 11.9527 - acc: 0.2541 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 2/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.08

KeyboardInterrupt: 